# Initialisation

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
import numpy as np
#import scipy
from scipy.special import zeta
from time import time
from astropy import cosmology as cosmo
from astropy import units as u
from astropy import constants as c
from astropy.visualization import quantity_support
quantity_support()

# Data

## Model-independent distances

In [ ]:
SNe_z, SNe_err_z, SNe_mu, SNe_err_mu = np.loadtxt('data/Pantheon+_Data.dat', usecols=(2, 3, 10, 11), unpack=True, skiprows=1)
GRB1 = np.loadtxt('data/GRB_RNN.dat', skiprows=1)
GRB2 = np.loadtxt('data/GRB_RNN_2.dat', skiprows=1)
GRB_z = np.concatenate([GRB1[:, 0], GRB1[:, 3], GRB1[:, 6], GRB2[:, 0]])
GRB_mu = np.concatenate([GRB1[:, 1], GRB1[:, 4], GRB1[:, 7], GRB2[:, 1]])
GRB_err_mu = np.concatenate([GRB1[:, 2], GRB1[:, 5], GRB1[:, 8], GRB2[:, 2]])
QSO_z, QSO_mu, QSO_err_mu = np.loadtxt('data/table3QSO.dat', usecols=(3, 11, 12), unpack=True, skiprows=1)
#QSO_err_mu *= 2

def Dl_Mpc(mu):
    return 1e-5 * 10**(.2*mu)

#SNe_Dl = Dl_Mpc(SNe_mu)
#GRB_Dl = Dl_Mpc(GRB_mu)
#QSO_Dl = Dl_Mpc(QSO_mu)


all_z = np.concatenate([SNe_z, QSO_z, GRB_z])
all_mu = np.concatenate([SNe_mu, QSO_mu, GRB_mu])
all_err_mu = np.concatenate([SNe_err_mu, QSO_err_mu, GRB_err_mu])

#all_z = SNe_z
#all_mu = SNe_mu
#all_err_mu = SNe_err_mu
#all_z = QSO_z
#all_mu = QSO_mu
#all_err_mu = QSO_err_mu
#all_z = GRB_z
#all_mu = GRB_mu
#all_err_mu = GRB_err_mu


In [ ]:
thetaBAO = np.array([
    0.11,19.8,3.26,
    0.235,9.06,0.23,
    0.365,6.33,0.22,
    0.45,4.77,0.17,
    0.47,5.02,0.25,
    0.49,4.99,0.21,
    0.51,4.81,0.17,
    0.53,4.29,0.30,
    0.55,4.25,0.25,
    0.57,4.59,0.36,
    0.59,4.39,0.33,
    0.61,3.85,0.31,
    0.63,3.90,0.43,
    0.65,3.55,0.16,
    2.225,1.77,0.31])

theta_z = thetaBAO[::3]
theta_z_err = np.array([0.005,0.035,0.025,0.01,0.005,0.01,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.01,0.025])
theta_angle_deg = thetaBAO[1::3]
theta_angle_err = thetaBAO[2::3]
#theta_relative_err = theta_angle_err / theta_angle_deg
#theta_DM = 180/np.pi/theta_angle_deg

In [ ]:
CCH_z, CCH_H, CCH_H_err = np.loadtxt('data/Favale23_CCH.txt', usecols=(0, 1, 3), unpack=True, skiprows=1)

## BAO based on fiducial ($\Lambda$CDM) cosmology

In [ ]:
# SDSS BAO-only (Table 3 in Alam et al. 2021)

sdss_z_DV = np.array([0.15, 0.85])
sdss_z_DV_err = np.array([[0.08, 0.25], [0.05, 0.25]])

sdss_DV_rd = np.array([4.47, 18.33])
sdss_DV_rd_err = np.array([0.17, 0.62])


sdss_z_DHDM = np.array([0.38, 0.51, 0.70, 1.48, 2.33, 2.33])
sdss_z_DHDM_err = np.array([[0.18, 0.11, 0.10, 0.68, 0.23, 0.56],
                            [0.12, 0.09, 0.30, 0.72, 1.67, 1.67]])

sdss_DH_rd = np.array([25, 22.33,  19.33,  13.26,  8.93, 9.08 ])
sdss_DH_rd_err = np.array([0.76, 0.58, 0.53, 0.55, 0.28, 0.34])

sdss_DM_rd = np.array([10.23, 13.36, 17.86, 30.69, 37.6, 37.3])
sdss_DM_rd_err = np.array([0.17, 0.21, 0.33, 0.8, 1.9, 1.7])
sdss_DA_rd = sdss_DM_rd / (1+sdss_z_DHDM)
sdss_DA_rd_err = sdss_DM_rd_err / (1+sdss_z_DHDM)


# DR7
'''
rfid7 = 153.5 #Mpc, r fiducial del SDSSDR7
DR7_z = np.array([0.275])
DR7_DV = np.array([7.2])
errDR7_DV = DR7_DV**2*np.array([0.0037])
xerrDR7_DV = np.array([[0.265],[0.225]])
'''


In [ ]:
# DES YR3 (Chan et al. 2022); first column in Table II (Gaussian)
DESyr3_z = np.array([0.75, 0.85, 0.95, 1.05])
DESyr3_z_err = np.array([0.05, 0.05, 0.05, 0.05])
DESyr3_DM_fid = cosmo.FlatLambdaCDM(H0=67.6, Om0=0.31).angular_diameter_distance(DESyr3_z)*(1+DESyr3_z)
DESyr3_r_fid = 147.6 * u.Mpc

DESyr3_DM_rd = DESyr3_DM_fid/DESyr3_r_fid * np.array([0.973, 0.960, 0.987, 0.891])
DESyr3_DM_rd_err = DESyr3_DM_fid/DESyr3_r_fid * np.array([0.037, 0.045, 0.039,0.019])
DESyr3_DA_rd = DESyr3_DM_rd / (1+DESyr3_z)
DESyr3_DA_rd_err = DESyr3_DM_rd_err / (1+DESyr3_z)

# DES YR6 (Abbott et al. 2024)
DESyr6_z = np.array([0.85])
DESyr6_z_err = np.array([[0.25], [0.35]])

DESyr6_DM_rd = np.array([19.51])
DESyr6_DM_rd_err = np.array([0.41])
DESyr6_DA_rd = DESyr6_DM_rd / (1+DESyr6_z)
DESyr6_DA_rd_err = DESyr6_DM_rd_err / (1+DESyr6_z)


In [ ]:
# 6dFGS (Beutler et al. 2011)

z_6df = np.array([0.106])
errz_6df = np.array([[0.096],[0.134]])
DV_6df = np.array([456])
DV_err_6df = np.array([27])

r_DV_6df = np.array([0.336])
r_DV_err_6df = np.array([0.015])


In [ ]:
# WiggleZ

# Blake er al. 2011, Table 1)

wiggleZ_z = np.array([0.44, 0.6, 0.73])
wiggleZ_z_err = np.array([[0.24, 0.2, 0.13], [0.16, 0.2, 0.27]])

cc = cosmo.FlatLambdaCDM(H0=71., Om0=0.27)
fid_DA = cc.angular_diameter_distance(wiggleZ_z).to_value(u.Mpc)
fid_DH = (c.c*wiggleZ_z / cc.H(wiggleZ_z)).to_value(u.Mpc)
fid_DV = np.power((1+wiggleZ_z)**2 * fid_DA**2 * fid_DH, 1/3)
print(f'DV_fid = {np.array2string(fid_DV, precision=2, floatmode="fixed")} (H0={cc.H0:.2f} Om0={cc.Om0:.3f})')
B11_DV_fid = np.array([1617.8, 2085.4, 2421.9])
print(f'DV_fid = {np.array2string(B11_DV_fid, precision=2, floatmode="fixed")}')

B11_r_DV = np.array([0.0916, 0.0726, 0.0592])
B11_r_DV_err = np.array([0.0071, 0.0034, 0.0032])

B11_DV = np.array([1.024, 1.003, 1.113]) * B11_DV_fid
B11_DV_err = np.array([0.093, 0.065, 0.071]) * B11_DV_fid
print(f'r_d = {B11_r_DV * B11_DV} Mpc is not constant :^(')


# WiggleZ (Kazin et al. 2014)
'''
K14_z = np.array([0.44, 0.6, 0.73])
K14_z_err = np.array([[0.24, 0.2, 0.13], [0.16, 0.2, 0.27]])
K14_r_fid = 152.3  # not used
K14_r_fid = 148.6  # adopted value
#K14_DV_rd = np.array([1716.4, 2220.8, 2516.1]) / K14_r_fid
K14_DV = np.array([1716.4, 2220.8, 2516.1])
K14_DV_fid = np.array([1617.7, 2085.2, 2421.7])
#
#K14_DV_rd = np.array([1.065, 1.001, 1.057]) * K14_DV_fid/K14_r_fid  # no reconstruction
K14_DV_rd = np.array([1.061, 1.065, 1.039]) * K14_DV_fid/K14_r_fid  # with reconstruction
#
K14_DV_err = np.array([83.1, 100.6, 86.1])
K14_DV_rd_err = K14_DV_err / K14_r_fid
'''


# Hinton er al. 2017, Table 1

cc = cosmo.FlatLambdaCDM(H0=70.5, Om0=0.273)
fid_DA = cc.angular_diameter_distance(wiggleZ_z).to_value(u.Mpc)
fid_DH = (c.c*wiggleZ_z / cc.H(wiggleZ_z)).to_value(u.Mpc)
fid_DV = np.power((1+wiggleZ_z)**2 * fid_DA**2 * fid_DH, 1/3)
#print(f'DV_fid = {np.array2string(fid_DV, precision=2, floatmode="fixed")} (H0={cc.H0:.2f} Om0={cc.Om0:.3f})')

H17_DV = np.array([1.07, 1.00, 1.10]) * fid_DV
H17_DV_err = np.array([[0.09, 0.08, 0.10], [0.13, 0.09, 0.09]]) * fid_DV[np.newaxis, :]

# Hinton et al. 2017, Table 3

H17_DA = np.array([1330, 1280, 1340])
H17_DA_err = np.array([[150, 160, 130], [150, 190, 150]])
H17_H = np.array([85, 91, 80])
H17_H_err = np.array([[12, 14, 10], [19, 15, 9]])

# Hinton et al. 2017, Table 4

H17_z = np.array([0.6, 0.73])
H17_z_err = [[0.2, 0.13],[0.2, 0.27]]

H17_DA_rd = np.array([10.3, 9.8])
H17_DA_rd_err = np.array([[0.5, 0.4], [0.4, 1.1]])
H17_DM_rd = (1+H17_z) * H17_DA_rd
H17_DM_rd_err = (1+H17_z) * H17_DA_rd_err

H17_DH_r = 1/H17_z * np.array([11.5, 15.3])
H17_DH_r_err = 1/H17_z * np.array([[1.6, 1.8],[1.3, 2.1]])


In [ ]:
#combinamos todos para el fit a D_V
#zvfit = np.concatenate((sdss_z,wigglez_z,z6df,DR7_z))
#DVfit = np.concatenate((DVDATA,DVwig,DV6df,DR7_DV))
#errDVfit = np.concatenate((errDVDATA,errDVwig,errDV6df,errDR7_DV))

# Model grid

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from models import FlatLCDM, Coasting

In [ ]:
def fit(model):
    chi2 = []
    chi2.append(model.fit_mu(SNe_z, SNe_mu, SNe_err_mu))
    chi2.append(model.fit_mu(QSO_z, QSO_mu, QSO_err_mu))
    chi2.append(model.fit_mu(GRB_z, GRB_mu, GRB_err_mu))
    
    chi2.append(model.fit_DA(wiggleZ_z, H17_DA, H17_DA_err))
    
    chi2.append(model.fit_H(CCH_z, CCH_H, CCH_H_err))
    chi2.append(model.fit_H(wiggleZ_z, H17_H, H17_H_err))
    
    chi2.append(model.fit_DV_rd(sdss_z_DV, sdss_DV, sdss_DV_err))
    chi2.append(model.fit_rd_DV(wiggleZ_z, B11_r_DV, B11_r_DV_err))
    chi2.append(model.fit_rd_DV(z_6df, r_DV_6df, r_DV_err_6df))
    
    chi2.append(model.fit_DA_rd(DESyr6_z, DESyr6_DA_rd, DESyr6_DA_rd_err))
    chi2.append(model.fit_DA_rd(DESyr3_z, DESyr3_DA_rd, DESyr3_DA_rd_err))
    chi2.append(model.fit_DA_rd(sdss_z_DHDM, sdss_DA_rd, sdss_DA_rd_err))
    chi2.append(model.fit_DA_rd(H17_z, H17_DA_rd, H17_DA_rd_err))
    chi2.append(model.fit_theta_BAO(theta_z, theta_angle_deg, theta_angle_err))
    
    chi2.append(model.fit_DH_rd(sdss_z_DHDM, sdss_DH_rd, sdss_DH_rd_err))
    chi2.append(model.fit_DH_rd(H17_z, H17_DH_r, H17_DH_r_err))

    return chi2

n_steps_H = 36
n_steps_Omega = 43
#n_steps_H = 6
#n_steps_Omega = 7
chi2 = np.empty((2, n_steps_H, n_steps_Omega, 16))

#H_steps = np.linspace(62, 82, n_steps_H) * u.km/u.s/u.Mpc
#Omega_L = np.linspace(0, .95, n_steps_Omega)
#Omega_k = np.linspace(-.95, 0, n_steps_Omega)
H_steps = np.linspace(35, 105, n_steps_H) * u.km/u.s/u.Mpc
Omega_L = np.linspace(-1.15, 0.95, n_steps_Omega)
Omega_k = np.linspace(-1.55, 0.55, n_steps_Omega)
#Omega_k = -np.logspace(-2, .3, n_steps_Omega)

t0 = time()
print('Creating grid...')
for i, H0 in enumerate(H_steps):
    print(H0, i, n_steps_H)
    
    for j, OL in enumerate(Omega_L):
        model = FlatLCDM(OL, H0)
        chi2[0, i, j] = fit(model)
    
    for j, Ok in enumerate(Omega_k):
        model = Coasting(Ok, H0)
        chi2[1, i, j] = fit(model)

print(f'... Done ({time()-t0:.1f} s)')

# Fitting

In [ ]:
chi2_best = np.nanmin(chi2, axis=(1, 2))

In [ ]:
xy = np.vstack((np.meshgrid(Omega_L, H_steps), np.meshgrid(Omega_k, H_steps)))


def plot_tracer(ax, model, tracer, color, label):
    ax.contour(xy[2*model], xy[2*model+1], chi2[model, :, :, tracer], levels=[0, chi2_best[model, tracer]+1], colors=[color], alpha=1.)
    ax.contourf(xy[2*model], xy[2*model+1], chi2[model, :, :, tracer], levels=[0, chi2_best[model, tracer]+1], colors=[color], alpha=.125)
    best_H, best_Omega = np.unravel_index(np.nanargmin(chi2[model, :, :, tracer].ravel()), (n_steps_H, n_steps_Omega))
    ax.scatter(xy[2*model, 0, :][best_Omega], xy[2*model+1, :, 0][best_H], color=color, label=label)
    weight = np.exp(-.5 * (chi2[model, :, :, tracer] - chi2_best[model, tracer])**2)
    weight /= np.nansum(weight)
    # WARNING: let's assume linear spacing!
    final_Omega = np.nansum(weight * xy[2*model])
    sigma_Omega = np.sqrt(np.nansum(weight * (xy[2*model] - final_Omega)**2))
    final_H = np.nansum(weight * xy[2*model+1])
    sigma_H = np.sqrt(np.nansum(weight * (xy[2*model+1] - final_H)**2))
    ax.errorbar(final_Omega, final_H, yerr=sigma_H, xerr=sigma_Omega, fmt='.', c=color)

## Hubble-Lemaître, DA, H

In [ ]:
fig_name = 'chi2-map'

plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(7, 9))
axes = fig.subplots(nrows=3, ncols=3, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0, 'width_ratios':(1, 1, .3)},
                   )

def plot_model(model):
    plot_tracer(axes[0, model], model, 0, 'r', 'SNe')
    plot_tracer(axes[0, model], model, 1, 'orange', 'QSO')
    plot_tracer(axes[0, model], model, 2, 'm', 'GRB')
    
    plot_tracer(axes[1, model], model, 3, 'g', r'WiggleZ')
    
    plot_tracer(axes[2, model], model, 4, 'c', 'CCH')
    plot_tracer(axes[2, model], model, 5, 'g', r'WiggleZ')


plot_model(0)
plot_model(1)

for panel in range(axes.shape[0]):
    handles, labels = axes[panel, 0].get_legend_handles_labels()
    axes[panel, 2].legend(handles, labels, loc='upper left')
    axes[panel, 2].axis('off')

axes[0, 0].set_title(r'Flat $\Lambda$CDM')
axes[0, 1].set_title('Coasting')

axes[0, 0].set_ylabel(r'$H_0$ [km/s/Mpc]')
axes[1, 0].set_ylabel(r'$H_0$ [km/s/Mpc]')
axes[2, 0].set_ylabel(r'$H_0$ [km/s/Mpc]')

axes[-1, 0].set_xlabel(r'$\Omega_\Lambda$')
axes[-1, 1].set_xlabel(r'$\Omega_k$')

for ax in axes.flatten():
    ax.tick_params(top=True, right=True, axis='both', which='both', direction='inout')
    ax.grid(alpha=.25)
fig.set_tight_layout(True)
plt.show()

## BAO

In [ ]:
fig_name = 'chi2-BAO'

plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(7, 9))
axes = fig.subplots(nrows=3, ncols=3, squeeze=False,
                    sharex='col', sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0, 'width_ratios':(1, 1, .3)},
                   )

def plot_model(model):
    plot_tracer(axes[0, model], model, 6, 'r', 'SDSS')
    plot_tracer(axes[0, model], model, 7, 'g', 'WiggleZ')
    plot_tracer(axes[0, model], model, 8, 'b', '6dFGS')

    plot_tracer(axes[1, model], model, 9, 'b', 'DES YR6')
    plot_tracer(axes[1, model], model, 10, 'c', 'DES YR3')
    plot_tracer(axes[1, model], model, 11, 'r', 'SDSS')
    plot_tracer(axes[1, model], model, 12, 'g', 'WiggleZ')
    plot_tracer(axes[1, model], model, 13, 'y', r'$\theta_{BAO}$')

    plot_tracer(axes[2, model], model, 14, 'r', 'SDSS')
    plot_tracer(axes[2, model], model, 15, 'g', 'WiggleZ')


plot_model(0)
plot_model(1)

for panel in range(axes.shape[0]):
    handles, labels = axes[panel, 0].get_legend_handles_labels()
    axes[panel, 2].legend(handles, labels, loc='upper left')
    axes[panel, 2].axis('off')

axes[0, 0].set_title(r'Flat $\Lambda$CDM')
axes[0, 1].set_title('Coasting')

axes[0, 0].set_ylabel(r'$H_0$ [km/s/Mpc]')
axes[1, 0].set_ylabel(r'$H_0$ [km/s/Mpc]')
axes[2, 0].set_ylabel(r'$H_0$ [km/s/Mpc]')

axes[-1, 0].set_xlabel(r'$\Omega_\Lambda$')
axes[-1, 1].set_xlabel(r'$\Omega_k$')

for ax in axes.flatten():
    ax.tick_params(top=True, right=True, axis='both', which='both', direction='inout')
    ax.grid(alpha=.25)
fig.set_tight_layout(True)
plt.show()

# Compare with observations

## Hubble-Lemaître, DA, H

In [ ]:
from models import flcdm, coasting

def plot_DL_DH_obs(ax, z, mu, err_mu, **kwargs):
    '''
    ax.errorbar(z, mu, err_mu, fmt='.', **kwargs)
    '''
    #scale = 3e5 * z
    #scale = z
    scale = (1 + z)**2
    y = Dl_Mpc(mu) / scale
    dy_low = y-Dl_Mpc(mu-err_mu) / scale
    dy_high = Dl_Mpc(mu+err_mu) / scale - y
    ax.errorbar(z, y, yerr=(dy_low, dy_high), fmt='.', **kwargs)
    print('chi^2 =', flcdm.fit_mu(z, mu, err_mu), coasting.fit_mu(z, mu, err_mu))
    
    
fig_name = 'observations_independent'

plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(7, 9))
axes = fig.subplots(nrows=3, ncols=1, squeeze=False,
                    sharex=True, sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )

ax = axes[0, 0]
#ax.set_ylabel(r'$D_L/z$ [Mpc]')
#ax.set_ylim(0, 2.1e4)
ax.set_ylabel(r'$D_L(z)/(1+z)^2$ [Mpc]')
#ax.set_ylim(0, 2400)
#ax.set_ylabel(r'$\mu$')
#ax.set_ylim(22, 58)
ax.set_ylim(3, 6000)
ax.set_yscale('log')

plot_DL_DH_obs(ax, SNe_z, SNe_mu, SNe_err_mu, c='r', alpha=.05, label='Pantheon+ SN Ia (Brout et al. 2022)')
plot_DL_DH_obs(ax, QSO_z, QSO_mu, QSO_err_mu, c='orange', alpha=.15, label='QSO (Lusso et al. 2020)')
plot_DL_DH_obs(ax, GRB_z, GRB_mu, GRB_err_mu, c='m', alpha=.25, label='GRB (Escamilla-Rivera et al. 2022)')


ax.plot(flcdm.z_t, flcdm.Dm_t.to_value(u.Mpc) / (1+flcdm.z_t), 'k--', label=r'Flat $\Lambda$CDM')
ax.plot(coasting.z_t, coasting.Dm_t.to_value(u.Mpc) / (1+coasting.z_t), 'k-', label='Coasting')
#ax.plot(flcdm.z_t, 5*np.log10(flcdm.Dm_t*(1+flcdm.z_t)/10/u.pc), 'k--', label=r'Flat $\Lambda$CDM')
#ax.plot(coasting.z_t, 5*np.log10(coasting.Dm_t*(1+coasting.z_t)/10/u.pc), 'k-', label='Coasting')

leg = ax.legend()
for lh in leg.legendHandles: 
    lh.set_alpha(1)
    
ax = axes[1, 0]
ax.set_ylabel(r'$D_A(z)$ [Mpc]')

ax.errorbar(wiggleZ_z, H17_DA, yerr=H17_DA_err, xerr=wiggleZ_z_err, fmt='.', color='g', label='WiggleZ (Hinton et al. 2017)')

ax.plot(flcdm.z_t, flcdm.Dm_t.to_value(u.Mpc) / (1+flcdm.z_t), 'k--', label=r'Flat $\Lambda$CDM')
ax.plot(coasting.z_t, coasting.Dm_t.to_value(u.Mpc) / (1+coasting.z_t), 'k-', label=r'Coasting')

ax.legend()


ax = axes[2, 0]
ax.set_ylabel(r'$H(z)/(1+z)$ [km/s/Mpc]')
ax.set_ylim(35, 105)
#ax.set_yscale('log')

ax.errorbar(CCH_z, CCH_H/(1+CCH_z), CCH_H_err/(1+CCH_z), color='c', fmt='.', label=r'CCH (Favale et al. 2023)')
ax.errorbar(wiggleZ_z, H17_H/(1+wiggleZ_z), yerr=H17_H_err/(1+wiggleZ_z), xerr=wiggleZ_z_err, fmt='.', color='g', label='WiggleZ (Hinton et al. 2017)')

ax.plot(flcdm.z_t, (flcdm.H_t / (1+flcdm.z_t)).to_value(u.km/u.s/u.Mpc), 'k--', label=r'Flat $\Lambda$CDM')
ax.plot(coasting.z_t, (coasting.H_t / (1+coasting.z_t)).to_value(u.km/u.s/u.Mpc), 'k-', label=r'Coasting')

print(flcdm.fit_H(CCH_z, CCH_H, CCH_H_err),
      coasting.fit_H(CCH_z, CCH_H, CCH_H_err))

ax.legend()


ax.set_xlabel('redshift $z$')
ax.set_xscale('log')
ax.set_xlim(1e-3, 20)
#ax.set_xlim(-.2, 8.2)

for ax in axes.flatten():
    ax.tick_params(top=True, right=True, axis='both', which='both', direction='inout')
    ax.grid(alpha=.25)
fig.set_tight_layout(True)
#fig.suptitle('Cosmology-independent probes')
plt.show()
#plt.savefig('DL.pdf')


## BAO

In [ ]:
from models import flcdm, coasting, p18

fig_name = 'observations_fiducial'

plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(7, 9))
axes = fig.subplots(nrows=3, ncols=1, squeeze=False,
                    sharex=True, sharey='row',
                    gridspec_kw={'hspace': 0, 'wspace': 0},
                   )


ax = axes[0, 0]
ax.set_ylabel(r'$D_V(z)/r_d$')
ax.set_ylim(1.5, 21)

ax.errorbar(sdss_z_DV, sdss_DV_rd, yerr=sdss_DV_rd_err, xerr=sdss_z_DV_err, fmt='.', color='r', label='SDSS (Alam et al. 2021)')
ax.errorbar(wiggleZ_z, 1/B11_r_DV, yerr=B11_r_DV_err/B11_r_DV**2, xerr=wiggleZ_z_err, fmt='.', color='g', label='WiggleZ (Blake et al. 2011)')
ax.errorbar(z_6df, 1/r_DV_6df, yerr=r_DV_err_6df/r_DV_6df**2, xerr=errz_6df, fmt='b.', label='6dFGS (Beutler et al. 2011)')

#ax.plot(p18.z_t, np.power(p18.Dm_t**2 * c.c*p18.z_t/p18.H_t, 1/3) / p18.r_d, 'k:', label=r'Planck18')
ax.plot(flcdm.z_t, np.power(flcdm.Dm_t**2 * c.c*flcdm.z_t/flcdm.H_t, 1/3) / flcdm.r_d, 'k--', label=r'Flat $\Lambda$CDM')
ax.plot(coasting.z_t, np.power(coasting.Dm_t**2 * c.c*coasting.z_t/coasting.H_t, 1/3) / coasting.r_d, 'k-', label=r'Coasting')

ax.legend()


ax = axes[1, 0]
ax.set_ylabel(r'$D_A(z)/r_d$')
ax.set_ylim(6.5, 13.5)

ax.errorbar(DESyr6_z, DESyr6_DA_rd, yerr=DESyr6_DA_rd_err, xerr=DESyr6_z_err, fmt='.', color='b', label='DES YR6 (Abbott et al. 2024)')
ax.errorbar(DESyr3_z, DESyr3_DA_rd, yerr=DESyr3_DA_rd_err, xerr=DESyr3_z_err, fmt='.', color='c', label='DES YR3 (Chan et al. 2022)')
ax.errorbar(sdss_z_DHDM, sdss_DA_rd, yerr=sdss_DA_rd_err, xerr=sdss_z_DHDM_err, fmt='.', color='r', label='SDSS (Alam et al. 2021)')
ax.errorbar(H17_z, H17_DA_rd, yerr=H17_DA_rd_err, xerr=H17_z_err, fmt='.', color='g', label='WiggleZ (Hinton et al. 2017)')
rad2deg = 180/np.pi
ax.errorbar(theta_z, rad2deg/theta_angle_deg, yerr=rad2deg*theta_angle_err/theta_angle_deg**2, xerr=theta_z_err, color='y', fmt='.', label=r'$\theta_{BAO}$ (Nunes et al. 2020)')

ax.plot(flcdm.z_t, flcdm.Dm_t / flcdm.r_d /(1+flcdm.z_t), 'k--', label=r'Flat $\Lambda$CDM')
ax.plot(coasting.z_t, coasting.Dm_t / coasting.r_d/(1+coasting.z_t), 'k-', label=r'Coasting')

ax.legend()


ax = axes[2, 0]
ax.set_ylabel(r'$D_H(z)/r_d$')
ax.set_ylim(7, 28)

ax.errorbar(sdss_z_DHDM, sdss_DH_rd, yerr=sdss_DH_rd_err, xerr=sdss_z_DHDM_err, fmt='.', color='r', label='SDSS (Alam et al. 2021)')
ax.errorbar(H17_z, H17_DH_r, yerr=H17_DH_r_err, xerr=H17_z_err, fmt='.', color='g', label='WiggleZ (Hinton et al. 2017)')

#ax.plot(p18.z_t, c.c/p18.H_t/p18.r_d, 'k:', label=r'Planck18')
ax.plot(flcdm.z_t, c.c/flcdm.H_t/flcdm.r_d, 'k--', label=r'Flat $\Lambda$CDM')
ax.plot(coasting.z_t, c.c/coasting.H_t/coasting.r_d, 'k-', label=r'Coasting')

ax.legend()


ax.set_xlabel('redshift $z$')
ax.set_xlim(0, 2.6)
#ax.set_xscale('log')
#ax.set_xlim(1e-3, 20)
#ax.set_xlim(-.2, 8.2)

for ax in axes.flatten():
    ax.tick_params(top=True, right=True, axis='both', which='both', direction='inout')
    ax.grid(alpha=.25)
fig.set_tight_layout(True)
#fig.suptitle(r'BAO with $\Lambda$CDM as fiducial cosmology')
plt.show()
#plt.savefig('DL.pdf')
